# Template Model Development Notebook

The instructions in this notebook will guide you to setup your ML project, and help you connect the pieces together for the automated deployment workflow between this ML repository and Algorithmia.

## 1. Train, evaluate and test your ML model
First train, evaluate and test your model as usual.

In [ ]:
# Import Python packages, make sure your requirements.txt contains them!

# Use your train/test data

# Create your model object

# Do train/eval/test as usual

## 2. Save your model object to a file
When your model is ready, save it locally. Expected model file name by the Github Action is **`model.pkl`** by default. If your model file is named differently, make sure to put it in your **`./github/workflows/algorithmia_deploy.yml`** 

You do not need to check-in your model file to this repository. During the workflow, this notebook will be executed on a Github worker machine and the resulting model file will be uploaded to Algorithmia by our Github Action.

In [ ]:
# An example snippet to save your model file would be:
joblib.dump(model, "model.pkl", compress=True)

## 3. Test your serving (Algorithm) code
Rename the **`YOUR_ALGORITHM_NAME.py`** file with your algorithm's name and start working on your model serving code. 

You can execute that script from within this notebook to test your code locally. 

When you run your .py file with the `%run` macro as below, it will begin its execution through the **`if __name__ == "__main__"`** line. Make sure that your code executes fine and your algorithm is ready to be pushed to Algorithmia.


In [ ]:
%run  YOUR_ALGORITHM_DIR/src/YOUR_ALGORITHM_NAME.py

## Final checks before committing this notebook

- Once you are happy with your tests, you can remove the **`if __name__ == "__main__"`** snippet from your algorithm script. Remember that all of its contents will be pushed to Algorithmia. Don't stress though, you can always edit!

- Make sure that you do not commit/push your Algorithmia API Key!